<a href="https://colab.research.google.com/github/brysoncar/group-project-1-working/blob/main/Module_21_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# I am going to bin the ASK_AMT column


# Define the categories for binning
categories = ['0', '1-9999', '10000-24999', '25000-99999', '100000-499999', '1M-5M', '10M-50M', '5M-10M', '50M+']

# Define the bin edges based on the categories
bin_edges = [0, 1, 10000, 25000, 100000, 1000000, 5000000, 10000000, 50000000, float('inf')]

# Bin the ASK_AMT column using the specified categories and bin edges
application_df['ASK_AMT_Binned'] = pd.cut(application_df['ASK_AMT'], bins=bin_edges, labels=categories, right=False)

# Display the DataFrame with the new binned column
print(application_df[['ASK_AMT', 'ASK_AMT_Binned']].head())

   ASK_AMT ASK_AMT_Binned
0     5000         1-9999
1   108590  100000-499999
2     5000         1-9999
3     6692         1-9999
4   142590  100000-499999


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
updated_application_df = application_df.drop(columns=['EIN', 'NAME','STATUS','USE_CASE'])
updated_application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,ASK_AMT_Binned
0,T10,Independent,C1000,Association,0,N,5000,1,1-9999
1,T3,Independent,C2000,Co-operative,1-9999,N,108590,1,100000-499999
2,T5,CompanySponsored,C3000,Association,0,N,5000,0,1-9999
3,T3,CompanySponsored,C2000,Trust,10000-24999,N,6692,1,1-9999
4,T3,Independent,C1000,Trust,100000-499999,N,142590,1,100000-499999


In [ ]:
# Determine the number of unique values in each column.
unique_values = updated_application_df.nunique()
unique_values

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
ORGANIZATION                 4
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
ASK_AMT_Binned               8
dtype: int64

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
application_value_counts = updated_application_df['APPLICATION_TYPE'].value_counts()
application_value_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T17', 'T15', 'T29', 'T14','T25','T2','T12','T13','T9']

# Replace in dataframe
for app in application_types_to_replace:
    updated_application_df['APPLICATION_TYPE'] = updated_application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
updated_application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts for binning
classification_value_counts = updated_application_df['CLASSIFICATION'].value_counts()
classification_value_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_value_counts_g_1 = updated_application_df['CLASSIFICATION'].value_counts()
classification_value_counts_g_1 = classification_value_counts_g_1[classification_value_counts_g_1 > 1]
print(classification_value_counts_g_1)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = ['C4000', 'C5000', 'C1270', 'C2700', 'C2800', 'C7100', 'C1300', 'C1280', 'C1230', 'C1400', 'C7200', 'C2300', 'C1240', 'C8000', 'C7120', 'C1500', 'C1800', 'C6000', 'C1250', 'C8200', 'C1238', 'C1278', 'C1235', 'C1237', 'C7210', 'C2400', 'C1720', 'C4100', 'C1257', 'C1600', 'C1260', 'C2710', 'C0', 'C3200', 'C1246', 'C1234', 'C1256', 'C1267', 'C1728', 'C1732', 'C1900', 'C1820', 'C6100', 'C1248', 'C2600', 'C1370', 'C5200', 'C2561', 'C1245', 'C4200', 'C2190', 'C4500', 'C8210', 'C1236', 'C4120', 'C2570', 'C1580', 'C3700', 'C2170', 'C2500', 'C1570', 'C1283', 'C2380', 'C2150']


# Replace in dataframe
for cls in classifications_to_replace:
    updated_application_df['CLASSIFICATION'] = updated_application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
updated_application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1197
C7000      777
C1700      287
Name: CLASSIFICATION, dtype: int64

In [ ]:
updated_application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,ASK_AMT_Binned
0,T10,Independent,C1000,Association,0,N,5000,1,1-9999
1,T3,Independent,C2000,Co-operative,1-9999,N,108590,1,100000-499999
2,T5,CompanySponsored,C3000,Association,0,N,5000,0,1-9999
3,T3,CompanySponsored,C2000,Trust,10000-24999,N,6692,1,1-9999
4,T3,Independent,C1000,Trust,100000-499999,N,142590,1,100000-499999


In [ ]:

# Convert categorical data to numeric with pd.get_dummies
dummies_affiliation = pd.get_dummies(updated_application_df['AFFILIATION'])
#dummies_use_case = pd.get_dummies(updated_application_df['USE_CASE'])
dummies_organization = pd.get_dummies(updated_application_df['ORGANIZATION'])
dummies_application_type = pd.get_dummies(updated_application_df['APPLICATION_TYPE'])
dummies_classification = pd.get_dummies(updated_application_df['CLASSIFICATION'])
dummies_income_amt = pd.get_dummies(updated_application_df['INCOME_AMT'])
dummies_special_considerations = pd.get_dummies(updated_application_df['SPECIAL_CONSIDERATIONS'])
dummies_ask_amt_binned = pd.get_dummies(updated_application_df['ASK_AMT_Binned'])

# Concatenate dummy variables with the original DataFrame
embedded_application_df = pd.concat([
    updated_application_df.drop(columns=['AFFILIATION', 'APPLICATION_TYPE','ORGANIZATION', 'CLASSIFICATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS','ASK_AMT_Binned']),
    dummies_affiliation,
    #dummies_use_case,
    dummies_organization,
    dummies_application_type,
    dummies_classification,
    dummies_income_amt,
    dummies_special_considerations,
    dummies_ask_amt_binned
], axis=1)



In [ ]:
#Change for optiization - Remove some features I do not think are important - and test
for column_name in embedded_application_df.columns:
    print(column_name)

ASK_AMT
IS_SUCCESSFUL
CompanySponsored
Family/Parent
Independent
National
Other
Regional
Association
Co-operative
Corporation
Trust
Other
T10
T19
T3
T4
T5
T6
T7
T8
C1000
C1200
C1700
C2000
C2100
C3000
C7000
Other
0
1-9999
10000-24999
100000-499999
10M-50M
1M-5M
25000-99999
50M+
5M-10M
N
Y
0
1-9999
10000-24999
25000-99999
100000-499999
1M-5M
10M-50M
5M-10M
50M+


In [ ]:
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,ASK_AMT_Binned
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1,1-9999
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1,100000-499999
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,1-9999
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1,1-9999
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1,100000-499999


In [ ]:
application_value_counts = updated_application_df['INCOME_AMT'].value_counts()
application_value_counts

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [ ]:
#This was a test step, the model worked better with the other categories, keeping code but commenting out
#tested removing special considerations
#Tested removing application types - no improvement
#Testing removing classifications now - push
#testing removing affiliation - definitely keep
#testing dropping income amount of 0 and USE_CASE
#keep organization


#Step 1 - Get rid of the test categories
embedded_application_df = embedded_application_df.drop(columns=['0'])

# Verify the DataFrame after removing the columns
for column_name in embedded_application_df.columns:
    print(column_name)

ASK_AMT
IS_SUCCESSFUL
CompanySponsored
Family/Parent
Independent
National
Other
Regional
Association
Co-operative
Corporation
Trust
Other
T10
T19
T3
T4
T5
T6
T7
T8
C1000
C1200
C1700
C2000
C2100
C3000
C7000
Other
1-9999
10000-24999
100000-499999
10M-50M
1M-5M
25000-99999
50M+
5M-10M
N
Y
1-9999
10000-24999
25000-99999
100000-499999
1M-5M
10M-50M
5M-10M
50M+


In [ ]:
#find new number of features:
num_columns = embedded_application_df.shape[1]
print("Number of columns in the DataFrame:", num_columns)

Number of columns in the DataFrame: 47


In [ ]:
# Split our preprocessed data into our features and target arrays

# Separate the target variable
target = embedded_application_df['IS_SUCCESSFUL']

# Extract features (excluding the target variable)
features = embedded_application_df.drop(columns=['IS_SUCCESSFUL'])

# Split the preprocessed data into a training and testing dataset

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

#Change #1 = Activation Method testing, after many different variations the one that worked best is below
#Change #2 = Add three more hidden layers
#Change #3 = Increasing the number of units in the hidden layers

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=128, activation="relu", input_dim=46))

# Second hidden layer
nn.add(Dense(units=128, activation='relu'))

# Third hidden layer (example of adding an additional hidden layer)
nn.add(Dense(units=64, activation='leaky_relu'))

#Fourth hidden layer (example of adding another hidden layer)
nn.add(Dense(units=64, activation='ELU'))

#Fifth hidden layer (example of adding another hidden layer)
nn.add(Dense(units=16, activation='leaky_relu'))

#Sixth hidden layer (example of adding another hidden layer)
nn.add(Dense(units=16, activation='relu'))

#Seventh hidden layer (example of adding another hidden layer)
nn.add(Dense(units=16, activation='ELU'))

#Eighth hidden layer (example of adding another hidden layer)
nn.add(Dense(units=16, activation='relu'))


# Output layer
nn.add(Dense(units=1, activation='sigmoid'))


# Check the structure of the model
nn.summary()

Model: "sequential_77"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_437 (Dense)           (None, 128)               6016      
                                                                 
 dense_438 (Dense)           (None, 128)               16512     
                                                                 
 dense_439 (Dense)           (None, 64)                8256      
                                                                 
 dense_440 (Dense)           (None, 64)                4160      
                                                                 
 dense_441 (Dense)           (None, 16)                1040      
                                                                 
 dense_442 (Dense)           (None, 16)                272       
                                                                 
 dense_443 (Dense)           (None, 16)              

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Compile the model with RMSprop optimizer
#nn.compile(loss="binary_crossentropy", optimizer="rms", metrics=["accuracy"])


In [ ]:
 # Fit the model to the training data
fit_model = nn.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
858/858 [==============================] - 5s 4ms/step - loss: 0.5668 - accuracy: 0.7180
Epoch 2/200
858/858 [==============================] - 2s 3ms/step - loss: 0.5543 - accuracy: 0.7285
Epoch 3/200
858/858 [==============================] - 3s 3ms/step - loss: 0.5513 - accuracy: 0.7306
Epoch 4/200
858/858 [==============================] - 2s 3ms/step - loss: 0.5497 - accuracy: 0.7323
Epoch 5/200
858/858 [==============================] - 3s 3ms/step - loss: 0.5477 - accuracy: 0.7324
Epoch 6/200
858/858 [==============================] - 3s 3ms/step - loss: 0.5477 - accuracy: 0.7330
Epoch 7/200
858/858 [==============================] - 2s 3ms/step - loss: 0.5467 - accuracy: 0.7329
Epoch 8/200
858/858 [==============================] - 2s 3ms/step - loss: 0.5460 - accuracy: 0.7337
Epoch 9/200
858/858 [==============================] - 2s 3ms/step - loss: 0.5450 - accuracy: 0.7352
Epoch 10/200
858/858 [==============================] - 3s 4ms/step - loss: 0.5447 - accura

In [ ]:
 # Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 1s - loss: 0.6223 - accuracy: 0.7198 - 1s/epoch - 6ms/step
Loss: 0.6223226189613342, Accuracy: 0.7198250889778137
